In [1]:
import os
import sys
import time
import pprint
from collections import Counter, defaultdict
sys.path.append('/home/psavine/source/viper')
sys.path.append("../")


import numpy as np
import importlib
import matplotlib.pyplot as plt

import spec.hvac as thvac
import src.viper as viper

import src.geom as geom
import lib.geo as geo
from src.geomType import GeomType
from scipy import spatial
import src.ui.visual as viz3


from trimesh.path.path import Path
from src.ui import visual
from itertools import chain
from src.ui.visual import meshcat
import shapely.geometry as sg
import trimesh
import pandas as pd
import matplotlib
import math
import spec.scenario as scene
import src.geombase as geombase

RL = importlib.reload
RL(visual)
VIZ = viz3.VIZ

_dir = '/home/psavine/source/viper/data/sample/'


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [2]:
jsond = scene.load_from_json('/home/psavine/source/viper/data/sample/test_sk2.json')

RL(thvac)
train_mesh = thvac.TestTrain.preprocess_(jsond)
print(len(train_mesh))
categories = {x.category for i, x in enumerate(train_mesh)}
categories = {x:i for i, x in enumerate(categories)}
train_ids = {x.id:x for x in train_mesh}
train_ixs = {x.id:i for i, x in enumerate(train_mesh)}
vs2 = viper.IndexSystem()
# ixrs = vs2(train_mesh)

4439


In [17]:
[VIZ['mesh/' +x].delete() 
 for x in {x.category for i, x in enumerate(train_mesh)}]
r = [viz3.viz_mesh(x, handle='mesh/' + x.meta['category'], opacity=0.4) 
     for x in train_mesh]
no = []
for p in train_mesh:
    if p.category == 'Pipes':
        if geombase.is_cylinder(p) == False:
            no.append(p)
print(len(no))

[viz3.viz_mesh(x, handle='no', opacity=0.8) for x in no]

1


[None]

In [18]:
V.delete('cadhead')
V.delete('line')

In [3]:
import dxfgrabber
dxf = dxfgrabber.readfile(_dir + 'WGFPR07m.dxf')
bodies = list(filter(lambda x: isinstance(x, dxfgrabber.dxfentities.Body), dxf.entities))
lines = list(filter(lambda x: isinstance(x, dxfgrabber.dxfentities.Line), dxf.entities))
inserts = list(filter(lambda x: isinstance(x, dxfgrabber.dxfentities.Insert), dxf.entities))

In [4]:
import src.formats.sat as sab
RL(sab)
reader = sab.SABReader()
acis_ents = [reader.read_single(x.acis) for x in bodies]

In [5]:
CAD_SCALE = 1 /  25.39999999999999858 
ANGLE = 346.9658914 
np.radians(360 - ANGLE)

0.227488110132509

In [6]:
def as_line(this, k='point'):
    pts = this.children_of_type(k)
    if len(pts) == 2:
        p1, p2 = pts
        return geo.Line(geo.Point(p1.geom), geo.Point(p2.geom))

    
def as_linexf(this, trs=np.array([0.,0.,0.]), scale=1.):
    pts = this.children_of_type('point')
    if len(pts) == 2:
        xf = this.children_of_type('transform')
        if len(xf) == 1:
            try:
                p1, p2 = pts
                p11 = (xf[0].apply(p1.geom) *  scale) + trs
                p12 = (xf[0].apply(p2.geom) *  scale) + trs
                return geo.Line(geo.Point(p11), geo.Point(p12))
            except Exception as e:
                return None


def dxf_line_to_line(dxline, bsxf=np.array([0.,0.,0.])):
    p1, p2 = [np.array(x) - bsxf for x in [dxline.start, dxline.end]] 
    return geo.Line(geo.Point(p1), geo.Point(p1))


def pull_points(acis_blocks):
    for i, acis_ents in enumerate(acis_blocks):
        pts = list(filter(lambda x: isinstance(x, sab.ACEllipse), acis_ents))
        if len(pts) == 2:
            p1, p2 = pts
            viz3.viz_line(p1.geom, p2.geom, handle='cad/{}'.format(i))


def viz_pts(pts, handle, clear=True, **kwargs):
    if clear is True:
        VIZ[handle].delete()
    for i, p in enumerate(pts):
        viz3.viz_point(p, handle=handle+'/{}'.format(i), **kwargs)

In [7]:
CAD_TRANSLATE = -1* np.array(list(as_linexf(acis_ents[0], scale=CAD_SCALE).points[0].coordinates))
# CAD_TRANSLATE[-1] = 0
print(CAD_TRANSLATE) 


[-15.4164401  312.02305601 -44.87684516]


In [8]:
# RL(viz3)
def viz_nodes(ndict):
    if isinstance(ndict, dict):
        nlist = list(ndict.values())
    else:
        nlist = ndict
    viz3.viz_nodes(nlist, 
                   handle='nodes', 
                   thickness=0.08, 
                   opacity=0.6, 
                   clear=True)

def viz_head():
    ddd = [viz3.viz_point(np.array(x.insert) - bsxf,  
                          handle='cadhead/{}'.format(i), 
                          radius=3, 
                          opacity=0.5) 
           for i, x in enumerate(sprks)]
    

def viz_sprink_to_edge(sbests, sprinks):
    VIZ['headlink'].delete()
    for i, (spr_ix, ngh) in enumerate(sbests.items()):
        if ngh is None:
            continue
        g1, g2 = ngh.geom
        pt3d = geo.Point(sprinks[spr_ix])
        cp = geo.Point(g1).midpoint_to(geo.Point(g2))
        viz3.viz(geo.Line(pt3d, cp), handle='headlink/{}'.format(i))


# Create Final Lines and Visualize

In [9]:
pipes_lines = list(filter(None, 
                          [as_linexf(x, 
                                     scale=CAD_SCALE, 
                                     trs=CAD_TRANSLATE) 
                           for x in acis_ents]))

pipes_long = [x for x in pipes_lines if 0.2 < x.length ]
print(len(pipes_long))
print(pipes_long[0].numpy)
mv = -1 * pipes_long[0].numpy[0]

VIZ['line'].delete()
viz3.viz_by_index(pipes_long, handle='line', thickness=0.05)

2387
[[ 26.59407129 -57.14410531  -2.82960107]
 [ 26.59407129 -57.14410531  -3.09042784]]


## read inserts and visualize

In [9]:
sprinklers = [x for x in inserts if x.name == '3DM_UPR']
print('num sprinklers' , len(sprinklers))
# print(mv)
print(CAD_TRANSLATE)
sample = np.array(sprinklers[2].insert)
print(sample)
sfn = lambda x: CAD_TRANSLATE  + (np.asarray(x.insert) * CAD_SCALE)

sprinklers2 = np.array(list(map(sfn, sprinklers)))
sprinklers2[:, -1] = 0
print(sprinklers2[2])

viz_pts(sprinklers2, 'cadhead', clear=True, radius=0.2, opacity=0.5)

num sprinklers 1403
[-15.4164401  312.02305601 -44.87684516]
[  463.1242295  -8199.00927198  1126.13126295]
[  2.81679729 -10.77258462   0.        ]


In [10]:
from scipy.spatial.distance import cdist 
import src.formats.skansk as sks
RL(sks)

def edge_with_id(root_node, eid):
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, True):
            if e.id == eid:
                return e

def node_with_id(nd, eid):
    for n in nd.__iter__(fwd=True, bkwd=True):
        if n.id == eid:
            return n

        
def apply_fn(root_node, fn, edges=True):
    seen = set()
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, edges):
            if e.id not in seen:
                seen.add(e.id)
                res = fn(e)
                if res is not None:
                    yield e

                    
def apply_node_edge_fn(root_node, fn):
    seen = set()
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, True):
            if e.id not in seen:
                seen.add(e.id)
                res = fn(n, e)
                if res is not None:
                    yield res           
        
        
def apply_node_fn(root_node, fn):
    apply_fn(root_node, fn, edges=False)
                    
        
def wrap_yield(fn):
    def run_x(x):
        res = fn(x)
        if res is True:
            return x
        else:
            return None
    return run_x

def viz_edge_attr(root_node, k, **kwargs):
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, True):
            if  e.get(k) is None:
                continue
            g1, g2 = e.geom
            # pipe = str(e.get('is_pipe', None))
            viz3.viz_line(g1, g2, handle=viz3.as_path(k,  e.id), **kwargs)


def viz_node_attr(root_node, k, v=None, clear=True, **kwargs):
    if clear is True:
        VIZ[k].delete()
    for n in root_node.__iter__(True, True):
        if n.get(k) is None:
            continue
        if v is not None and n.get(k) != v:
            continue
        viz3.viz_point(n.geom, handle=viz3.as_path(k,  n.id), **kwargs)
            
            
            
def as_linecyl(this, trs=np.array([0.,0.,0.]), scale=1.):
    line = (this.line.numpy * scale) + trs
    
    rads = this.radius * scale
    this.line = geo.Line(geo.Point(line[0]), geo.Point(line[1]))
    this.radius = rads
    return this


def fix_triangles(root_node_, result):
    for lres in result:
        tris = [edge_with_id(root_node_, x) for x in lres]
        tris = [x for x in tris if x is not None]
        if len(tris) == 3:
            sks.resolve_triangle(tris)
        


In [11]:
from lib.geo import Point, Line
RL(sab)
reader = sab.SABReader()
acis_ents2 = list(filter(lambda x: x.valid is True,
                       map(sab.SACCylinder,
                           acis_ents)))
                           #map(lambda x: reader.read_single(x.acis),
                           #    bodies))))
print(len(acis_ents2))


14427


In [12]:
pipes_cyl = list(
    filter(lambda x: x.line.length > x.radius,
    filter(lambda x: x and x.valid is True, 
        map(lambda x: as_linecyl(x,scale=CAD_SCALE, trs=CAD_TRANSLATE) 
                           ,acis_ents2))))
print(len(pipes_cyl))

8208


In [13]:
def xfrom(node, scale=1., translate=None, **kwargs):
    nd = np.array(list(node.geom)) * scale
    if translate is not None:
        nd += translate
    node.geom = sks.tuplify(nd)
    return node

In [14]:
import src.formats.skansk as sks
import src.propogate as gp
from src.structs import Node, Edge
RL(sks)
RL(gp)
import src.ui.visual as V

# V.viz_order(rn1, handle='build')
# RL(viz3)
"""
--------------------------------------------------
SAVE THESE SETTINGS. 

FOR FUCKS SAKE DO NOT DELETE !!!!!!!!!!!!!!!

FILTER = 0.3
CONNECT = 0.4
--------------------------------------------------
"""
CAD_ANGLE = np.radians(360 - ANGLE)
def build_roots():
    RL(sks)
    FILTER = 0.3
    CONNECT = 0.4
    
    pipes_long = [x for x in pipes_cyl if x.line.length > FILTER]
    # print('filtered pipes:', len(pipes_long))

    # Build the Graph
    pairs2, tree, pts_ind = sks.connect_by_pairs(pipes_long, factor=CONNECT)
    node_dict2 = sks.make_node_dict(pairs2, pts_ind, pipes_long)
    node_list, comps_starts = sks.connected_components(list(node_dict2.values()))
    comps_starts = np.asarray(comps_starts)
    root_nodes = [node_list[i] for i in comps_starts]
    # print('final lengths', len(node_list), len(comps_starts))
    return root_nodes


In [150]:
def viz_kif(root_, k, i, **kwargs):
    # print(k, kwargs.get(k, None))
    if kwargs.get(k, None) is True:
        viz3.viz_iter(root_, handle='grp/{}/{}'.format(i, k), 
                      clear=True, opacity=0.3)
    else:
        viz3.delete(k)

shows = dict(heads=True, edge=True, 
             before=True, buildo=True, tris=True, move=True)

def xprocess(_Nodes, ix=0, v=True, **kwargs):
    # Find Logical Root
    RL(sks)
    RL(gp)
    base_root_node = _Nodes[ix]
    SR = sks.SpatialRoot()
    SR.__call__(base_root_node)
    # viz_kif(base_root_node, 'before', ix, **kwargs)
    print(SR.best)
    
    Root = node_with_id(base_root_node, SR.best)

    Buildo = gp.BuildOrder()
    Mover = sks.Rotator(CAD_ANGLE)
    Elbower = sks.DetectElbows()
    Triangler = sks.DetectTriangles(build=True)
    
    # Direct the edges 
    Root = gp.EdgeDirector()(Root)
    Root = Buildo(Root)

    tree = sks.resolve_heads(Root, sprinklers2)
    # viz_kif(base_root_node, 'heads', ix, **kwargs)
        
    # Apply Transformts - translate, scale rotate
    Root = Mover(Root)
    # viz_kif(base_root_node, 'move', ix, **kwargs)
    # V.viz_iter(Root, handle='grp/{}/pre'.format(ix), clear=True, opacity=0.4)
    
    Root = gp.EdgeDirector()(Root)
    Root = gp.BuildOrder()(Root)
    Root = Triangler(Root)
    Root = gp.EdgeDirector()(Root)
    
    
    # Root = sks.FunctionQ(sks.add_ups)(Root)
    Root = gp.GraphTrim()(Root, tol=0.01)
    # V.viz_iter(Root, handle='grp/{}/tri'.format(ix), clear=True, opacity=0.4)
    Root = gp.EdgeDirector()(Root)
    Root = sks.FunctionQ(sks.remove_short)(Root, lim=0.25)
    
    FirstEdges = Root.neighbors(edges=True)
    Elbower(FirstEdges)
    
    
    Root = gp.EdgeDirector()(Root)
    if v is True:
        viz3.viz_iter(Root, handle='grp/{}/final'.format(ix), 
                      clear=True, opacity=0.4)

        # viz_node_attr(Root, 'is_head', clear=True, radius=0.02)
    else:
        VIZ['grp/{}/final'.format(ix)].delete()
    return Root # , tree

In [179]:
RL(sks)
# RL(V)
root_nodes = build_roots()
[V.delete(x) for x in ['tee1', 'edge', 'tee2', 'tee3', 'long', 'short']]

rn0  = xprocess(root_nodes,  ix=0, v=True)
V.viz_order(rn0, clear=True)
viz_edge_attr(rn0, 'tap_edge', clear=True, radius=0.02)
# viz_node_attr(rn0, 'tee_mid', clear=True, radius=0.05)
# viz_node_attr(rn0, 'tee_index', clear=True, radius=0.03)

SpatialRoot seen nodes :  330
4014251
EdgeDirector seen nodes :  330
BuildOrder seen nodes :  330
KDTreeIndex seen nodes :  330
Rotator seen nodes :  239
EdgeDirector seen nodes :  239
BuildOrder seen nodes :  239
DetectTrianglesOld seen nodes :  229
EdgeDirector seen nodes :  211
GraphTrim seen nodes :  211
EdgeDirector seen nodes :  98
FunctionQ seen nodes :  91
DetectElbows seen nodes :  90
EdgeDirector seen nodes :  91


In [168]:
RL(V)
V.viz_heirarchical(rn0, 'htest', clear=True)
# TO_REVIT(rn0, 11)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


[]

In [114]:
import inspect
RL(src.propogate.propagators)
RL(src.propogate)
RL(gp)
inspect.getmro(gp.GraphTrim)

(src.propogate.propagators.GraphTrim,
 src.propogate.base.QueuePropogator,
 object)

In [180]:
RL(sks)
# rn0 = sks.LongestPath()(rn0)
TO_REVIT(rn0, 14)

EdgeDirector seen nodes :  91
Annotator seen nodes :  91
BuildOrder seen nodes :  91
17
90 71 16


In [174]:
V.viz_order(rn0, 'htest', clear=True)

----------------------------------------------

In [1493]:
# todo
def low_line(pts_index):
    lowest_ix = np.argsort(pts_index[:, -1])
    lowest_spr = pts_index[lowest_ix[0:2]]
    return Line(Point(lowest_spr[0]), Point(lowest_spr[1]))


CAD_Line = low_line(pts_ind)
p1, p2 = CAD_Line.points
rot = sks.Rotator(CAD_ANGLE)
cln = CAD_Line.moved(rot.M)


REV_Line = low_line(rvpts_index)

print(CAD_Line.length / REV_Line.length)

viz3.viz(REV_Line, handle='low/rev', thickness=1)
viz3.viz(cln, handle='low/cad', thickness=1)



0.4724310517187339


In [177]:
import src.render.render_propogators as rps
import src.process as process
import json
RL(rps)
RL(sks)
RL(process)
SHRINK = -0.05
OUTPATH = '/home/psavine/source/viper/data/out/sks/'

def save_outs(args, pth):
    geom, inds, syms = args
    res = {'geom': geom, 'indicies': inds, 'symbols': syms}
    print(len(geom), len(inds), len(syms))
    st = str(round(time.time(), 0))
    # data = json.dumps(res)
    with open('{}to-revit-{}.json'.format(OUTPATH, pth), 'w') as F:
        json.dump(res, F)
        
        
def _prepare(start, end, last=False):
    Shrink = -0.01
    edge = start.edge_to(end)
    radius = edge.get('radius')
    if radius is None:
        radius = 1.
    crv = geom.MepCurve2d(start.geom, end.geom)
    if last is False:
        p1, p2 = crv.extend(Shrink, Shrink).points
    else:
        p1, p2 = crv.extend(Shrink, 0).points

    if math.isnan(p1[0]):
        print(start, end, p1, p2)
    vec = list(p1) + list(p2) + [radius]
    return vec    


def _preparep(start, end, last=False):
    Shrink = -0.01
    edge = start.edge_to(end)
    radius = edge.get('radius')
    if radius is None:
        radius = 1.
    vec = list(start.geom) + list(end.geom) + [radius]
    return vec    


def TO_REVIT(root_nd, n, prep=_preparep):
    anno = sks.Annotator('$create', mapping={'has_head': 3})
    gp.EdgeDirector()(root_nd)
    anno(root_nd)

    gp.BuildOrder()(root_nd)

    print(anno._pos)
    res = process.finalize(root_nd, prep)

    save_outs(res, n)

def TO_REVIT2(root_nd, n):
    anno = sks.Annotator('$create', mapping={'has_head': 3})
    gp.EdgeDirector()(root_nd)
    anno(root_nd)

    gp.BuildOrder()(root_nd)

    print(anno._pos)
    res = process.finalize(root_nd, _prepare)

    save_outs(res, n)

EdgeDirector seen nodes :  292
Annotator seen nodes :  292
BuildOrder seen nodes :  292
0
291 180 0


In [153]:
RL(geombase)
pts = []
for msh in train_mesh:
    if msh.category =='Pipes':
        ln = geombase.as_cylinder(msh)
        if ln is not None:
            pts.append(ln.numpy)
rvpts_index = np.concatenate(pts)

In [178]:
import src.propogate.propagators as gpp
RL(gpp)

def buildo(node, value=-2, **kwargs):
    return node.get('order', -1) == value

def slope(crv):
    npp = crv.numpy
    # print(crv.length)
    z = npp[0, -1] - npp[1, -1]
    
    return np.abs(z / Line(*map(Point, crv.points)).length)



fedge = rn1.successors(edges=True)[0]

n10 = gpp.Where(buildo, edges=True)(fedge, value=10)
n11 = gpp.Where(buildo, edges=True)(fedge, value=11)
n8 = gpp.Where(buildo, edges=True)(fedge, value=8)

print(n8.similar_direction(n11))
print(n8.similar_direction(n10))
print(n10.similar_direction(n8))
# print(n10.direction(n8))

edges__ = [n10, n11, n8]

np.allclose(n10.direction, n10.direction)

[print(np.round(x.direction, 3)) for x in edges__]
preped = [_prepare(x.source, x.target) for x in edges__]
res = [preped[0][0:3], preped[1][0:3], preped[2][3:6]]
for i, r in enumerate(res):
    V.viz_point(r, handle='test/' + str(i), radius=0.04)

False
False
False
[-0.001  1.     0.013]
[0. 0. 1.]
[ 0.     1.    -0.008]


In [154]:

viz3.viz_point(rn1.geom, handle='test')

In [169]:
scs = rn1.successors(edges=True)
[x.curve.length for x in scs]
scs[1].target.id
Line(*map(Point, scs[1].curve.points)).length


0.25

In [152]:
x = [[print(x.curve.length) for x in n.successors(edges=True)] for n in rn1.__iter__()]

4.424903815802851
0.0
4.724409448892871
0.0
4.724416227465029
0.0
4.724402670148194
0.0
4.4881889763999965
0.0
1.4957284255232712
0.0
2.478527986093234
5.669292060701303
1.4957555886543963
4.682671853122132
4.488188976346123
0.0
4.724395891408776
0.0
4.724423006181383
0.0
4.72440944888227
0.0
4.424903412963972
0.0
0.067175219912541
0.9865987346366425
0.07236078501142551
2.1760124066530855
0.034878687351237996
26.190306445110405
4.717697120001906
0.9564337814230855
5.669293752049265
1.4958002112352013
1.4957740893192808
2.478518031069224
4.488188976443702
0.0
4.7244094488386565
0.0
4.724409448839083
0.0
4.724409448774793
0.0
4.424903789235771
0.0
4.488188976393822
0.0
4.724409448746627
0.0
4.724409448799719
0.0
4.724409448872876
0.0
4.42490342864997
0.0
5.669296178601118
1.4958606619651829
0.9363991779603119
1.4958965948616998
0.23363305708584128
0.37765748035644303
0.18956692907133119
0.37765748029841023
4.488188976393715
0.0
4.724409448788109
0.0
4.724409448798654
0.0
4.72440944882455

# --------------------------------------------------------------

In [1385]:
for i in range(len(comps_starts)):
    viz3.viz_iter(root_nodes[i], handle='nodec/{}'.format(i),
                  clear=True, opacity=0.3)

DetectTriangles seen nodes :  330   
DetectElbows seen nodes :  347   
elbows: 28 , tirangles 18   


In [1414]:
RL(sks)

RL(geo)
from lib.geo import Line, Point # , Movement
import lib.meshcat.src.meshcat.transformations as txf


def rotator(orienting_edge):
    rnt = np.array([orienting_edge.source.geom, 
                    orienting_edge.target.geom])

    rnt[:, -1] = 0

    norm = Line(Point(rnt[1]), Point(rnt[0]))
    print(norm)
    base = Line(Point(0., 0., 0.), Point(1., 0, 0))
    m = geo.Movement(norm, base)
    
    angle = norm.angle_to(base)
    print(angle)
    return m
    # return -txf.rotation_matrix(angle, [0.,0., 1], rnt[1])[:3, :3]
    

edg = edge_with_id(ROOT, 4438916)
arr = np.asarray([list(edg.source.geom), list(edg.target.geom)])
print(np.round(arr, 3))
print(edg.curve.length)
M = rotator(edg)

print(M.q, M.dr)
p1 = Point(edg.source.geom)
p2 = Point(edg.target.geom)
ln = Line(Point(edg.source.geom), Point(edg.target.geom))


md = ln.moved(M)
print(ln.length, md.length)



[[ 2.42334e+02  0.00000e+00 -1.26000e-01]
 [ 2.36876e+02  0.00000e+00 -1.78000e-01]]
5.4582469151953035
Line(Point(236.876, 0, 0), Point(242.334, 0, 0))
0.0
None [0. 0. 0.]
5.458492828173617 5.458492828173617


### Connect Heads

## MISC TESTING


In [1031]:
import src.propogate
# RL(src.propogate.base)
RL(sks)

root = edge_with_id(root_nodes[0], 1275321)
nd = root.source
print(root)
sks.EdgeDirectorQ()(nd)


<Edge>:((58.17832921906061, -128.39112719713268, 0.3211833299301574), (57.19465048862049, -124.14188319947925, 0.29480801374617727)), {'is_pipe': True}


In [1035]:
VIZ['nodec'].delete() 
viz3.viz_iter(nd, handle='nodec/0', 
                   opacity=0.3)  

len(list(nd.__iter__(fwd=True, bkwd=True)))

1158

In [357]:
sprinks_arr = sprinklers3
sprinks_2d = sprinks_arr[:, 0:2]
sprinks_tree = kdtree.KDTree(sprinks_arr[:, 0:2])

pipe_2d = np.concatenate(plines)[:, 0:2]
pipe_2dtree = kdtree.KDTree( pipe_2d)
tr = sprinks_tree.query_ball_tree(pipe_2dtree, 1)

sprinks_to_pipes = dict()
pipes_to_sprinks = defaultdict(set)
sprinks_to_pipes_d = dict()
for i, x in enumerate(tr):
    if len(x) > 0:
        dists, pipe_ixs = sprinks_tree.query(pipe_2d[x])
        sprinks_to_pipes[i] = x[np.argmin(dists)]
        pipes_to_sprinks[x[np.argmin(dists)]].add(i) 
        sprinks_to_pipes_d[i] = dists[np.argmin(dists)]


array([  391.57757842, -7925.38562256,  1139.87186712])

## TRY WITH WHOLE SAT

In [96]:
headers, data = lines[0:3], lines[3:]

In [110]:
def dec_sat(bin_line):
    return bin_line.decode('ascii').split(' ')


def recur_bod(ix):
    
    seen = set()
    others = []
    q = [ix]
    while q:
        this_ix = q.pop()
        # print(this_ix)
        if this_ix in seen:
            continue
            
        seen.add(this_ix)
        raw = data[this_ix]
        
        chunks = dec_sat(raw)
        others.append(chunks)
        print(this_ix, chunks)
        for c in chunks:
            if c[0] == '$' and c != '$-1':
                todo = int(c[1:])
                # print('todo', todo)
                q.append(todo) 

    return others
            

# data[108302: 108306]
recur_bod(1)

1 ['body', '$-1', '-1', '$-1', '$21463', '$-1', '$21464', '#\r\n']
21464 ['transform', '$-1', '-1', '1', '0', '0', '0', '1', '0', '0', '0', '1', '-1101197.235013285652', '109246.670983092743', '832.9999939502620236', '1', 'rotate', 'no_reflect', 'no_shear', '#\r\n']
21463 ['lump', '$-1', '-1', '$-1', '$-1', '$64713', '$1', '#\r\n']
64713 ['shell', '$-1', '-1', '$-1', '$-1', '$-1', '$86505', '$-1', '$21463', '#\r\n']
86505 ['face', '$108301', '-1', '$-1', '$-1', '$108302', '$64713', '$-1', '$108303', 'reversed', 'double', 'out', '#\r\n']
108303 ['\t0.1691308163470010006', '2', '0.171370211494747321', '3', '\r\n']
108302 ['\t0.1624126309037620397', '3', '0.1668914211992546803', '2', '0.1680111187731278544', '2', '0.1685709675600644275', '2', '0.1688508919535327002', '2', '\r\n']
108301 ['\t0', '2', '1', '2', '\r\n']


[['body', '$-1', '-1', '$-1', '$21463', '$-1', '$21464', '#\r\n'],
 ['transform',
  '$-1',
  '-1',
  '1',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '1',
  '-1101197.235013285652',
  '109246.670983092743',
  '832.9999939502620236',
  '1',
  'rotate',
  'no_reflect',
  'no_shear',
  '#\r\n'],
 ['lump', '$-1', '-1', '$-1', '$-1', '$64713', '$1', '#\r\n'],
 ['shell',
  '$-1',
  '-1',
  '$-1',
  '$-1',
  '$-1',
  '$86505',
  '$-1',
  '$21463',
  '#\r\n'],
 ['face',
  '$108301',
  '-1',
  '$-1',
  '$-1',
  '$108302',
  '$64713',
  '$-1',
  '$108303',
  'reversed',
  'double',
  'out',
  '#\r\n'],
 ['\t0.1691308163470010006', '2', '0.171370211494747321', '3', '\r\n'],
 ['\t0.1624126309037620397',
  '3',
  '0.1668914211992546803',
  '2',
  '0.1680111187731278544',
  '2',
  '0.1685709675600644275',
  '2',
  '0.1688508919535327002',
  '2',
  '\r\n'],
 ['\t0', '2', '1', '2', '\r\n']]